# Fabric: Highlights into AI/LLMs

In [2]:
!pip show synapseml

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 5, Finished, Available, Finished)

Name: synapseml
Version: 1.0.8
Summary: Synapse Machine Learning
Home-page: https://github.com/Microsoft/SynapseML
Author: Microsoft
Author-email: synapseml-support@microsoft.com
License: MIT
Location: /home/trusted-service-user/cluster-env/clonedenv/lib/python3.11/site-packages
Requires: 
Required-by: 


In [3]:
!pip show langchain-openai

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 6, Finished, Available, Finished)

Name: langchain-openai
Version: 0.2.4
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/trusted-service-user/cluster-env/clonedenv/lib/python3.11/site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


In [4]:
!pip show langchain

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 7, Finished, Available, Finished)

Name: langchain
Version: 0.3.6
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/trusted-service-user/cluster-env/clonedenv/lib/python3.11/site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


## Ensure you have the API key and endpoint URL for your deployed model. Set these as environment variables

In [19]:
import os

os.environ["OPENAI_API_VERSION"] = "2023-08-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = "your-value"

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 22, Finished, Available, Finished)

## Create an instance of the Azure OpenAI class using the environment variables set above

In [20]:
from langchain_openai import AzureChatOpenAI

# Set the API base URL
api_base = os.environ["AZURE_OPENAI_ENDPOINT"]

# Create an instance of the Azure OpenAI Class
llm = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    temperature=0.7,
    verbose=True,
    top_p=0.9
)

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 23, Finished, Available, Finished)

## Use the Azure OpenAI service to generate text or perform other language model tasks

In [21]:
# Define a prompt
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "Hi, how are you?"),
]

# Generate a response from the Azure OpenAI service using the invoke method
ai_msg = llm.invoke(messages)

# Print the response
print(ai_msg)

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 24, Finished, Available, Finished)

content='Salut, comment ça va ?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 33, 'total_tokens': 39, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_d54531d9eb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-8cb7f29a-44c1-4f65-a648-15afb2d793dc-0' usage_metadata={'input_tokens': 33, 'output_tokens': 6, 'total_tokens': 39, 'input_token_details': {}, 'output_token_details': {}}


## Define a prompt template for generating definitions

In [22]:
from langchain.prompts import PromptTemplate

copy_prompt = PromptTemplate(
    input_variables=["technology"],
    template="Define the following word: {technology}",
)

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 25, Finished, Available, Finished)

## Create an LLMChain with the defined prompt template

In [23]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=copy_prompt)


StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 26, Finished, Available, Finished)

## Set up the LangChain transformer to execute the processing chain


In [24]:
from synapse.ml.cognitive.langchain import LangchainTransformer

openai_api_key= os.environ["AZURE_OPENAI_API_KEY"]

transformer = (
    LangchainTransformer()
    .setInputCol("technology")
    .setOutputCol("definition")
    .setChain(chain)
    .setSubscriptionKey(openai_api_key)
    .setUrl(api_base)
)

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 27, Finished, Available, Finished)

## Construct a DataFrame with technology names.

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Initialize Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Construct a DataFrame with technology names
df = spark.createDataFrame(
    [
        (0, "docker"), (1, "spark"), (2, "python")
    ],
    ["label", "technology"]
)

# Define a simple UDF to transform the technology column
def transform_technology(tech):
    return tech.upper()

# Register the UDF
transform_udf = udf(transform_technology, StringType())

# Apply the UDF to the DataFrame
transformed_df = df.withColumn("transformed_technology", transform_udf(df["technology"]))

# Show the transformed DataFrame
transformed_df.show()

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 29, Finished, Available, Finished)

+-----+----------+----------------------+
|label|technology|transformed_technology|
+-----+----------+----------------------+
|    0|    docker|                DOCKER|
|    1|     spark|                 SPARK|
|    2|    python|                PYTHON|
+-----+----------+----------------------+



## Extract content from PDFs linked in arXiv papers and generate prompts for extracting specific information.


In [27]:
from langchain.document_loaders import OnlinePDFLoader

def paper_content_extraction(inputs: dict) -> dict:
    arxiv_link = inputs["arxiv_link"]
    loader = OnlinePDFLoader(arxiv_link)
    pages = loader.load_and_split()
    return {"paper_content": pages[0].page_content + pages[1].page_content}

def prompt_generation(inputs: dict) -> dict:
    output = inputs["Output"]
    prompt = (
        "find the paper title, author, summary in the paper description below, output them. "
        "After that, Use websearch to find out 3 recent papers of the first author in the author section below "
        "(first author is the first name separated by comma) and list the paper titles in bullet points: "
        "<Paper Description Start>\n" + output + "<Paper Description End>."
    )
    return {"prompt": prompt}

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 30, Finished, Available, Finished)

## Set up a chain to extract structured information from an arXiv link


In [28]:
from langchain.chains import TransformChain, SimpleSequentialChain

paper_content_extraction_chain = TransformChain(
    input_variables=["arxiv_link"],
    output_variables=["paper_content"],
    transform=paper_content_extraction,
    verbose=False,
)

paper_summarizer_template = """
You are a paper summarizer, given the paper content, it is your job to summarize the paper into a short summary, 
and extract authors and paper title from the paper content.
"""

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 31, Finished, Available, Finished)

## Use Microsoft Fabric's native integration with the MLflow framework to log the trained machine learning models, the used hyperparameters, and evaluation metrics.

In [30]:
import mlflow
from mlflow.models import infer_signature
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor

# Generate synthetic regression data
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

# Model parameters
params = {"n_estimators": 3, "random_state": 42}

# Model tags for MLflow
model_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023"
}

# Log MLflow entities
with mlflow.start_run() as run:
    # Train the model
    model = RandomForestRegressor(**params).fit(X, y)

    # Infer the model signature
    signature = infer_signature(X, model.predict(X))

    # Log parameters and the model
    mlflow.log_params(params)
    mlflow.sklearn.log_model(model, artifact_path="sklearn-model", signature=signature)

    # Register the model with tags
    model_uri = f"runs:/{run.info.run_id}/sklearn-model"
    model_version = mlflow.register_model(model_uri, "RandomForestRegressionModel", tags=model_tags)

    # Output model registration details
    print(f"Model Name: {model_version.name}")
    print(f"Model Version: {model_version.version}")

StatementMeta(, 7383b5d4-1dea-4b9b-85d6-fe5ef5b7d325, 33, Finished, Available, Finished)

Registered model 'RandomForestRegressionModel' already exists. Creating a new version of this model...
2024/10/31 04:36:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestRegressionModel, version 2
Created version '2' of model 'RandomForestRegressionModel'.


Model Name: RandomForestRegressionModel
Model Version: 2


## Use MLflow to search among multiple models saved within the workspace

In [ ]:
from pprint import pprint
from mlflow.tracking import MlflowClient

client = MlflowClient()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)